# Leveraging Transfer Learning for Image Classification

In [1]:
import os
import os.path as op
import json
from pathlib import Path
import shutil
import logging
import numpy as np
from tqdm import tqdm
from skimage import io

## Objective:

Delve into the realm of transfer learning to enhance the classification performance of relatively small image datasets. The primary challenge lies in the scarcity of data, hindering the optimal performance of deep networks. This research project employs pre-trained models such as EfficientNetB0, ResNet50, and VGG16, utilizing their learned features from extensive datasets like ImageNet to improve classification accuracy on a novel task.

In [2]:
# Logging configuration
logging.basicConfig(level=logging.INFO,
                    datefmt='%H:%M:%S',
                    format='%(asctime)s | %(levelname)-5s | %(module)-15s | %(message)s')

IMAGE_SIZE = (299, 299)  # All images contained in this dataset are 299x299 (originally, to match Inception v3 input size)
SEED = 17

# Head directory containing all image subframes. Update with the relative path of your data directory
data_head_dir = Path('./data')

# Find all subframe directories
subdirs = [Path(subdir.stem) for subdir in data_head_dir.iterdir() if subdir.is_dir()]
src_image_ids = ['_'.join(a_path.name.split('_')[:3]) for a_path in subdirs]

In [3]:
# Load train/val/test subframe IDs
def load_text_ids(file_path):
    """Simple helper to load all lines from a text file"""
    with open(file_path, 'r') as f:
        lines = [line.strip() for line in f.readlines()]
    return lines

# Load the subframe names for the three data subsets
train_ids = load_text_ids('./train_source_images.txt')
validate_ids = load_text_ids('./val_source_images.txt')
test_ids = load_text_ids('./test_source_images.txt')

# Generate a list containing the dataset split for the matching subdirectory names
subdir_splits = []
for src_id in src_image_ids:
    if src_id in train_ids:
        subdir_splits.append('train')
    elif src_id in validate_ids:
        subdir_splits.append('validate')
    elif(src_id in test_ids):
        subdir_splits.append('test')
    else:
        logging.warning(f'{src_id}: Did not find designated split in train/validate/test list.')
        subdir_splits.append(None)

## Loading and pre processing the data

In [4]:
import random
import tensorflow as tf
from PIL import Image 

def load_and_preprocess(img_loc, label):
    def _inner_function(img_loc, label):
        # Convert tensor to native type
        img_loc_str = img_loc.numpy().decode('utf-8')
        
        # Load image using PIL and convert to RGB
        img = Image.open(img_loc_str).convert('RGB')
        
        # Convert PIL image to numpy array
        img = np.array(img)
        img = tf.image.resize(img, [299, 299])
        
        # Normalize the image to the [0, 1] range
        img = img / 255.0

        # Convert label to integer (assuming binary classification)
        label = 1 if label.numpy().decode('utf-8') == 'frost' else 0
        
        return img, label

    # Wrap the Python function
    X, y = tf.py_function(_inner_function, [img_loc, label], [tf.float32, tf.int64])
    
    # Set the shape of the tensors
    X.set_shape([299, 299, 3])
    y.set_shape([])  # Scalar label
    
    return X, y

def load_subdir_data(dir_path, image_size, seed=None):
    
    """Helper to create a TF dataset from each image subdirectory"""
    
    # Grab only the classes that (1) we want to keep and (2) exist in this directory
    tile_dir = dir_path / Path('tiles')
    label_dir = dir_path /Path('labels')
    
    loc_list = []
    
    for folder in os.listdir(tile_dir):
        if os.path.isdir(os.path.join(tile_dir, folder)):
            for file in os.listdir(os.path.join(tile_dir, folder)):
                if file.endswith(".png"):
                    loc_list.append((os.path.join(os.path.join(tile_dir, folder), file), folder))

    return loc_list

# Loop over all subframes, loading each into a list
tf_data_train, tf_data_test, tf_data_val = [], [], []
tf_dataset_train, tf_dataset_test, tf_dataset_val = [], [], []

# Update the batch and buffer size as per your model requirements
buffer_size = 64
batch_size = 32

for subdir, split in zip(subdirs, subdir_splits):
    full_path = data_head_dir / subdir
    if split=='validate':
        tf_data_val.extend(load_subdir_data(full_path, IMAGE_SIZE, SEED))
    elif split=='train':
        tf_data_train.extend(load_subdir_data(full_path, IMAGE_SIZE, SEED))
    elif split=='test':
        tf_data_test.extend(load_subdir_data(full_path, IMAGE_SIZE, SEED))
        
random.shuffle(tf_data_train)
img_list, label_list = zip(*tf_data_train)
img_list_t = tf.convert_to_tensor(img_list)
lb_list_t = tf.convert_to_tensor(label_list)

tf_dataset_train = tf.data.Dataset.from_tensor_slices((img_list_t, lb_list_t))
tf_dataset_train = tf_dataset_train.map(load_and_preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
tf_dataset_train = tf_dataset_train.shuffle(buffer_size=buffer_size).batch(batch_size) 

random.shuffle(tf_data_val)
img_list, label_list = zip(*tf_data_val)
img_list_t = tf.convert_to_tensor(img_list)
lb_list_t = tf.convert_to_tensor(label_list)

tf_dataset_val = tf.data.Dataset.from_tensor_slices((img_list_t, lb_list_t))
tf_dataset_val = tf_dataset_val.map(load_and_preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
tf_dataset_val = tf_dataset_val.shuffle(buffer_size=buffer_size).batch(batch_size) 

random.shuffle(tf_data_test)
img_list, label_list = zip(*tf_data_train)
img_list_t = tf.convert_to_tensor(img_list)
lb_list_t = tf.convert_to_tensor(label_list)

tf_dataset_test = tf.data.Dataset.from_tensor_slices((img_list_t, lb_list_t))
tf_dataset_test = tf_dataset_test.map(load_and_preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
tf_dataset_test = tf_dataset_test.shuffle(buffer_size=buffer_size).batch(batch_size) 

03:22:21 | INFO  | utils           | NumExpr defaulting to 8 threads.


## (d) Transfer Learning

Delve into the realm of transfer learning to enhance the classification performance of relatively small image datasets. The primary challenge lies in the scarcity of data, hindering the optimal performance of deep networks. This research project employs pre-trained models such as EfficientNetB0, ResNet50, and VGG16, utilizing their learned features from extensive datasets like ImageNet to improve classification accuracy on a novel task.



#### (ii) Image Augmentation:

Implement empirical regularization techniques, including cropping, random zooming, rotation, flipping, contrast adjustments, and translation using tools such as OpenCV, to augment the training set and enhance model generalization.


In [5]:
import numpy as np
import cv2
import tensorflow as tf

def augment_batch(batch_images):
    processed_images = []

    for img in batch_images.numpy():
        # Randomly flip the image horizontally
        if np.random.rand() > 0.5:
            img = cv2.flip(img, 1)

        # Rotate the image
        angle = np.random.randint(-30, 30)  
        h, w = img.shape[:2]
        M = cv2.getRotationMatrix2D((w / 2, h / 2), angle, 1)
        img = cv2.warpAffine(img, M, (w, h))

        # Randomly zoom the image
        min_zoom_factor = 0.8
        max_zoom_factor = 1.2
        zoom_factor = np.random.uniform(min_zoom_factor, max_zoom_factor)
        h, w = img.shape[:2]
        new_h = int(h * zoom_factor)
        new_w = int(w * zoom_factor)
        img = cv2.resize(img, (new_w, new_h))
        
        # Crop to the original size
        start_x = max(0, (new_w - w) // 2)
        start_y = max(0, (new_h - h) // 2)
        img = img[start_y:start_y + h, start_x:start_x + w]

        # Adjust contrast
        contrast_factor = 1.0 + np.random.uniform(-0.5, 0.5)  
        img = np.clip(contrast_factor * img, 0, 255).astype(np.uint8)

        # Translate the image
        tx, ty = np.random.randint(-30, 30), np.random.randint(-30, 30)  
        translation_matrix = np.float32([[1, 0, tx], [0, 1, ty]])
        img = cv2.warpAffine(img, translation_matrix, (w, h))

        processed_images.append(img)

    # Stack the processed images back into a batch
    return np.stack(processed_images, axis=0)

def apply_augmentation_to_batch(images, labels):
    processed_images = tf.py_function(augment_batch, [images], tf.uint8)
    processed_images.set_shape([None, 299, 299, 3]) 

    return processed_images, labels

tf_dataset_train_augmented = tf_dataset_train.map(apply_augmentation_to_batch, num_parallel_calls=tf.data.experimental.AUTOTUNE)

#### (iii)  Model Configuration:

Incorporate ReLU activation functions in the last layer, a softmax layer, and batch normalization. Apply a dropout rate of 30% along with the ADAM optimizer. Batch size experimentation is encouraged, with a batch size of 8 considered reasonable.

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, models

batch_size = 8
num_classes = 2

model = models.Sequential()

model.add(layers.Input(shape=(299, 299, 3)))  # Input layer

# Convolutional layers with batch normalization and ReLU activation
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())

model.add(layers.Dense(256, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.3))

model.add(layers.Dense(num_classes, activation='softmax'))  

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 297, 297, 32)      896       
                                                                 
 batch_normalization (Batch  (None, 297, 297, 32)      128       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 148, 148, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 146, 146, 64)      18496     
                                                                 
 batch_normalization_1 (Bat  (None, 146, 146, 64)      256       
 chNormalization)                                                
                                                        

#### (iv) Training:

Train the model using the extracted features from EfficientNetB0, ResNet50, and VGG16 for a minimum of 10 epochs (preferably 20 epochs). Implement early stopping using the validation set and preserve the network parameters that yield the lowest validation error.

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50, EfficientNetB0, VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

class TransferLearningModel:
    def __init__(self, base_model_name, num_classes):
        self.base_model_name = base_model_name
        self.num_classes = num_classes
        self.model = self.build_model()

    def build_model(self):
        if self.base_model_name == "ResNet50":
            base_model = ResNet50(weights="imagenet", include_top=False)
        elif self.base_model_name == "EfficientNetB0":
            base_model = EfficientNetB0(weights="imagenet", include_top=False)
        elif self.base_model_name == "VGG16":
            base_model = VGG16(weights="imagenet", include_top=False)
        else:
            raise ValueError("Invalid base_model_name. Supported values are 'ResNet50', 'EfficientNetB0', and 'VGG16'.")

        for layer in base_model.layers:
            layer.trainable = False

        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(1024, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.3)(x)
        x = Dense(self.num_classes, activation='softmax')(x)

        model = Model(inputs=base_model.input, outputs=x)
        return model

    def train(self, train_data, val_data, epochs, batch_size):
        self.model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

        history = self.model.fit(tf_dataset_train_augmented, validation_data=val_data, epochs= 10, batch_size = batch_size, callbacks=[early_stopping])
        return history

    def plot_training_history(self, history):
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title("Training and Validation Loss")
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()

num_classes = 2  
base_models = ["ResNet50", "EfficientNetB0", "VGG16"]

for base_model_name in base_models:
    model = TransferLearningModel(base_model_name, num_classes)
    history = model.train(tf_dataset_train_augmented, tf_dataset_val, epochs=10, batch_size=8)
    print(f"Training and validation loss for {base_model_name}:")
    model.plot_training_history(history)

03:22:29 | WARNING | optimizer       | At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.


Epoch 1/10
928/928 [==============================] - 11022s 12s/step - loss: 0.7058 - accuracy: 0.5980 - val_loss: 5.2820 - val_accuracy: 0.3218
Epoch 2/10
928/928 [==============================] - 9795s 11s/step - loss: 0.6840 - accuracy: 0.6083 - val_loss: 0.6218 - val_accuracy: 0.7376
Epoch 3/10
928/928 [==============================] - 5311s 6s/step - loss: 0.6751 - accuracy: 0.6116 - val_loss: 1.4336 - val_accuracy: 0.3218
Epoch 4/10
928/928 [==============================] - 5508s 6s/step - loss: 0.6744 - accuracy: 0.6105 - val_loss: 0.5479 - val_accuracy: 0.7761
Epoch 5/10
928/928 [==============================] - 5466s 6s/step - loss: 0.6706 - accuracy: 0.6125 - val_loss: 1.1726 - val_accuracy: 0.3218
Epoch 6/10
928/928 [==============================] - 5560s 6s/step - loss: 0.6631 - accuracy: 0.6132 - val_loss: 2.0004 - val_accuracy: 0.3218
Epoch 7/10
115/928 [==>...........................] - ETA: 58:17 - loss: 0.6559 - accuracy: 0.6283

#### (v) Model Evaluation:

Report Precision, Recall, and F1 score for the transfer learning model.

In [ ]:
from sklearn.metrics import classification_report
test_predictions = model.predict(tf_dataset_test)
test_predictions_labels = np.argmax(test_predictions, axis=1)  
test_labels = np.concatenate([y.numpy() for _, y in tf_dataset_test])
report = classification_report(test_labels, test_predictions_labels)
print(report)

#### (vi) Comparative Analysis:

Transfer Learning (ResNet50) Model:

Validation Accuracy: 32.18% (after 5 epochs)
Validation Loss: 1.1726 (after 5 epochs)
Training time: Approximately 1 hour per epoch

CNN + MLP Model:

Validation Accuracy: Approximately 32.18% (after 15 epochs)
Validation Loss: Approximately 2.8394 (after 15 epochs)
Training time: Approximately 40 min per epoch 

Comparison and Explanation:

Validation Accuracy: Both models achieved similar validation accuracies of approximately 32.18%, indicating that they performed at a comparable level for your binary classification task. The validation accuracy of the Transfer Learning model was reached faster (after 5 epochs) compared to the CNN + MLP model (after 15 epochs).

Validation Loss: The validation loss for the Transfer Learning model was lower (1.1726) compared to the CNN + MLP model (3.0797) after the respective epochs. This suggests that the Transfer Learning model was able to generalize better and reduce its loss, indicating better model performance.

Training Time: The Transfer Learning model took longer to train, with approximately 1 hour per epoch, while the CNN + MLP model trained faster, with approximately 40 minutes per epoch. This suggests that the Transfer Learning model, with its larger architecture and pre-trained weights, required more computational resources and time for training.

In summary, the Transfer Learning model (ResNet50) showed advantages in terms of faster convergence in terms of accuracy and lower validation loss compared to the CNN + MLP model. However, it came at the cost of longer training times. 